# LAB | Abstractive Question Answering

Abstractive question-answering focuses on the generation of multi-sentence answers to open-ended questions. It usually works by searching massive document stores for relevant information and then using this information to synthetically generate answers. This notebook demonstrates how Pinecone helps you build an abstractive question-answering system. We need three main components:

- A vector index to store and run semantic search
- A retriever model for embedding context passages
- A generator model to generate answers

# Install Dependencies

In [6]:
!pip install datasets  datasets==2.16.1 sentence-transformers torch

# Load and Prepare Dataset

Our source data will be taken from the Wiki Snippets dataset, which contains over 17 million passages from Wikipedia. But, since indexing the entire dataset may take some time, we will only utilize 50,000 passages in this demo that include "History" in the "section title" column. If you want, you may utilize the complete dataset. Pinecone vector database can effortlessly manage millions of documents for you.

In [7]:
from datasets import load_dataset

# load the dataset from huggingface in streaming mode and shuffle it
wiki_data = load_dataset(
  'squad',
  split='train',
  streaming=True
).shuffle(seed=960)

We are loading the dataset in the streaming mode so that we don't have to wait for the whole dataset to download (which is over 9GB). Instead, we iteratively download records one at a time.

In [8]:
# show the contents of a single document in the dataset
next(iter(wiki_data))

{'id': '56bf8c8aa10cfb140055116f',
 'title': 'Beyoncé',
 'context': 'In July 2002, Beyoncé continued her acting career playing Foxxy Cleopatra alongside Mike Myers in the comedy film, Austin Powers in Goldmember, which spent its first weekend atop the US box office and grossed $73 million. Beyoncé released "Work It Out" as the lead single from its soundtrack album which entered the top ten in the UK, Norway, and Belgium. In 2003, Beyoncé starred opposite Cuba Gooding, Jr., in the musical comedy The Fighting Temptations as Lilly, a single mother whom Gooding\'s character falls in love with. The film received mixed reviews from critics but grossed $30 million in the U.S. Beyoncé released "Fighting Temptation" as the lead single from the film\'s soundtrack album, with Missy Elliott, MC Lyte, and Free which was also used to promote the film. Another of Beyoncé\'s contributions to the soundtrack, "Summertime", fared better on the US charts.',
 'question': "How did the critics view the movie

In [9]:
titles = [doc['title'] for doc in wiki_data]

In [10]:
from collections import Counter

title_counts = Counter(titles)

# Number of unique titles
num_unique_titles = len(title_counts)
print(f"Number of unique titles: {num_unique_titles}")

# Show top 10 most common titles
print(title_counts.most_common(40))

Number of unique titles: 442
[('New_York_City', 817), ('American_Idol', 802), ('Beyoncé', 758), ('Frédéric_Chopin', 697), ('Queen_Victoria', 680), ('Buddhism', 610), ('New_Haven,_Connecticut', 602), ('2008_Sichuan_earthquake', 521), ('2008_Summer_Olympics_torch_relay', 500), ('Muammar_Gaddafi', 489), ('Hellenistic_period', 469), ('Napoleon', 458), ('Middle_Ages', 452), ('Modern_history', 448), ('Portugal', 435), ('Gamal_Abdel_Nasser', 433), ('Dwight_D._Eisenhower', 430), ('Kanye_West', 428), ('Southampton', 426), ('The_Blitz', 414), ('Greece', 406), ('Religion_in_ancient_Rome', 405), ('Gramophone_record', 404), ('Dog', 392), ('Roman_Republic', 392), ('Pacific_War', 391), ('Financial_crisis_of_2007%E2%80%9308', 390), ('Mexico_City', 390), ('History_of_India', 389), ('Paris', 389), ('London', 379), ('Pub', 377), ('Tucson,_Arizona', 370), ('Protestantism', 369), ('Pharmaceutical_industry', 368), ('Plymouth', 367), ('Boston', 362), ('Age_of_Enlightenment', 361), ('Premier_League', 358), ('

In [11]:
# filter only documents with History as section_title - Replace None with your code
history = [doc for doc in wiki_data if 'History' in doc.get('title', '')]
print(f'Number of documents with "History" in title: {len(history)}')

Number of documents with "History" in title: 698


Let's iterate through the dataset and apply our filter to select the 50,000 historical passages. We will extract `article_title`, `section_title` and `passage_text` from each document.

In [12]:
from tqdm.auto import tqdm  # progress bar

total_doc_count = 50000

historical_passages = []

# Total number of documents for tqdm
# total_doc_count = len(wiki_data)

# Iterate through the dataset
for doc in tqdm(wiki_data, total=total_doc_count, desc="Filtering historical passages"):
    # Check if 'history' appears in the context (case-insensitive)
    if 'history' in doc.get('context', '').lower():
        historical_passages.append({
            'article_title': doc.get('title', ''),
            # 'section_title': 'History',       # Assign History as section
            'passage_text': doc.get('context', '')
        })
    # Stop once we have 50,000 passages
    if len(historical_passages) > 50000:
        break

Filtering historical passages:   0%|          | 0/50000 [00:00<?, ?it/s]

In [13]:
print(len(historical_passages))

3743


In [14]:
import pandas as pd

# create a pandas dataframe with the documents we extracted
df = pd.DataFrame(historical_passages)
df.head()

,article_title,passage_text
0,Beyoncé,"A self-described ""modern-day feminist"", Beyonc..."
1,Beyoncé,"In August, the couple attended the 2011 MTV Vi..."
2,Beyoncé,"In August, the couple attended the 2011 MTV Vi..."
3,Beyoncé,"On December 13, 2013, Beyoncé unexpectedly rel..."
4,University_of_Notre_Dame,One of the main driving forces in the growth o...


In [15]:
!pip install pinecone

# Initialize Pinecone Index

The Pinecone index stores vector representations of our historical passages which we can retrieve later using another vector (query vector). To build our vector index, we must first establish a connection with Pinecone. For this, we need an API from Pinecone. You can get one for free from [here](https://app.pinecone.io/), and after that, we initialize the connection as follows:

In [16]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
print(api_key)

# configure client
pc = Pinecone(api_key=api_key)

PINECONE_API_KEY


Now we setup our index specification, this allows us to define the cloud provider and region where we want to deploy our index. You can find a list of all [available providers and regions here](https://docs.pinecone.io/docs/projects).

In [17]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

Now we create a new index. We will name it "abstractive-question-answering" — you can name it anything we want. We specify the metric type as "cosine" and dimension as 768 because the retriever we use to generate context embeddings is optimized for cosine similarity and outputs 768-dimension vectors.

In [18]:
index_name = "abstractive-question-answering" #give your index a meaningful name

In [19]:
import time

dimension_size = 768
metric_type = "cosine"

# check if the extractive-question-answering index exists
if index_name not in pc.list_indexes().names():
    # create the index if it does not exist
    pc.create_index(
        name=index_name,
        dimension=dimension_size,
        metric=metric_type,
        # Specify Serverless configuration
        spec=spec
    )
# connect to extractive-question-answering index we created
index = pc.Index(index_name)

# Initialize Retriever

Next, we need to initialize our retriever. The retriever will mainly do two things:

- Generate embeddings for all historical passages (context vectors/embeddings)
- Generate embeddings for our questions (query vector/embedding)

The retriever will create embeddings such that the questions and passages that hold the answers to our queries are close to one another in the vector space. We will use a SentenceTransformer model based on Microsoft's MPNet as our retriever. This model performs quite well for comparing the similarity between queries and documents. We can use Cosine Similarity to compute the similarity between query and context vectors generated by this model (Pinecone automatically does this for us).

In [20]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
model_name = "flax-sentence-embeddings/all_datasets_v3_mpnet-base"

retriever = SentenceTransformer(model_name, device=device) #load the retriever model from HuggingFace. Use the flax-sentence-embeddings/all_datasets_v3_mpnet-base model
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'MPNetModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Generate Embeddings and Upsert

Next, we need to generate embeddings for the context passages. We will do this in batches to help us more quickly generate embeddings and upload them to the Pinecone index. When passing the documents to Pinecone, we need an id (a unique value), context embedding, and metadata for each document representing context passages in the dataset. The metadata is a dictionary containing data relevant to our embeddings, such as the article title, section title, passage text, etc.

In [21]:
# we will use batches of 64
batch_size = 64

batch_size = 64

all_vectors_to_upsert = [] # Moved initialization outside the loop to accumulate correctly

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    end = min(i + batch_size, len(df))
    # extract batch
    batch = df.iloc[i:end]
    # generate embeddings for batch
    contexts_to_embed = batch['passage_text'].tolist()
    emb = retriever.encode(contexts_to_embed).tolist()
    # get metadata
    meta = batch.to_dict(orient='records')
    meta_cleaned = []
    for record in meta:
        meta_cleaned.append({
            "passage_text": record['passage_text'],
            "article_title": record['article_title']
            # Add any other relevant fields like 'question', 'answer_text' if needed later
        })
    meta = meta_cleaned # Use the cleaned metadata
    # create unique IDs
    ids = [f"doc_{j}" for j in range(i, end)]
    # add all to upsert list
    to_upsert = []
    for doc_id, vector, metadata_dict in zip(ids, emb, meta):
        to_upsert.append({
            "id": doc_id,
            "values": vector,
            "metadata": metadata_dict
        })
    # Optional: collect all for debugging (keep this line indented)
    all_vectors_to_upsert.extend(to_upsert) # This accumulates data from all batches

    # upsert/insert these records to pinecone
    # The `index` object is what you got from `index = pc.Index(name)`
    _ = index.upsert(vectors=to_upsert) # <--- CORRECT INDENTATION: This call should be inside the loop for each batch


# check that we have all vectors in index
# These lines should be outside the loop to be run once after all upserts are complete
index_stats = index.describe_index_stats()
print(index_stats)

# verify the count from your DataFrame
print(f"Expected number of vectors (from df): {len(df)}")
index.describe_index_stats()

  0%|          | 0/59 [00:00<?, ?it/s]

{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '187',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 19 Nov 2025 10:26:16 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '36',
                                    'x-pinecone-request-id': '2647668170356290628',
                                    'x-pinecone-request-latency-ms': '36'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 3743}},
 'storageFullness': 0.0,
 'total_vector_count': 3743,
 'vector_type': 'dense'}
Expected number of vectors (from df): 3743


{'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '187',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 19 Nov 2025 10:26:16 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '5',
                                    'x-pinecone-request-id': '246885104984468355',
                                    'x-pinecone-request-latency-ms': '7'}},
 'dimension': 768,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 3743}},
 'storageFullness': 0.0,
 'total_vector_count': 3743,
 'vector_type': 'dense'}

# Initialize Generator

We will use ELI5 BART for the generator which is a Sequence-To-Sequence model trained using the ‘Explain Like I’m 5’ (ELI5) dataset. Sequence-To-Sequence models can take a text sequence as input and produce a different text sequence as output.

The input to the ELI5 BART model is a single string which is a concatenation of the query and the relevant documents providing the context for the answer. The documents are separated by a special token &lt;P>, so the input string will look as follows:

>question: What is a sonic boom? context: &lt;P> A sonic boom is a sound associated with shock waves created when an object travels through the air faster than the speed of sound. &lt;P> Sonic booms generate enormous amounts of sound energy, sounding similar to an explosion or a thunderclap to the human ear. &lt;P> Sonic booms due to large supersonic aircraft can be particularly loud and startling, tend to awaken people, and may cause minor damage to some structures. This led to prohibition of routine supersonic flight overland.

More detail on how the ELI5 dataset was built is available [here](https://arxiv.org/abs/1907.09190) and how ELI5 BART model was trained is available [here](https://yjernite.github.io/lfqa.html).

Let's initialize the BART model using transformers.

In [22]:
from transformers import BartTokenizer, BartForConditionalGeneration

# load bart tokenizer and model from huggingface
tokenizer = BartTokenizer.from_pretrained('vblagoje/bart_lfqa')
generator = BartForConditionalGeneration.from_pretrained('vblagoje/bart_lfqa').to(device)

All the components of our abstract QA system are complete and ready to be queried. But first, let's write some helper functions to retrieve context passages from Pinecone index and to format the query in the way the generator expects the input.

In [23]:
def query_pinecone(query, top_k):
    # generate embeddings for the query
    xq = retriever.encode(query).tolist()
    # search pinecone index for context passage with the answer
    xc =  xc = index.query(
        vector=xq,
        top_k=top_k,
        include_metadata=True
    )
    return xc


In [24]:
def format_query(query, context):
    # extract passage_text from Pinecone search result and add the <P> tag
    context = [f"<P> {m['metadata']['passage_text']}" for m in context]

    # concatinate all context passages
    context = " ".join(context)
    # contcatinate the query and context passages

    query = query+context
    print(query)
    return query

Let's test the helper functions. We will query the Pinecone index function we created earlier with the `query_pinecone` to get context passages and pass them to the `format_query` function.

In [44]:
query = "when was the first electric power system built?"
result = query_pinecone(query, top_k=1)
result

QueryResponse(matches=[{'id': 'doc_596',
 'metadata': {'article_title': 'Alexander_Graham_Bell',
              'passage_text': 'In 1880, Bell received the Volta Prize with a '
                              'purse of 50,000 francs (approximately '
                              "US$250,000 in today's dollars) for the "
                              'invention of the telephone from the Académie '
                              'française, representing the French government. '
                              'Among the luminaries who judged were Victor '
                              'Hugo and Alexandre Dumas. The Volta Prize was '
                              'conceived by Napoleon Bonaparte in 1801, and '
                              'named in honor of Alessandro Volta, with Bell '
                              'receiving the third grand prize in its history. '
                              'Since Bell was becoming increasingly affluent, '
                              'he used his prize 

In [45]:
from pprint import pprint

In [46]:
# format the query in the form generator expects the input
query = format_query(query, result["matches"])
# pprint(query)

when was the first electric power system built?<P> In 1880, Bell received the Volta Prize with a purse of 50,000 francs (approximately US$250,000 in today's dollars) for the invention of the telephone from the Académie française, representing the French government. Among the luminaries who judged were Victor Hugo and Alexandre Dumas. The Volta Prize was conceived by Napoleon Bonaparte in 1801, and named in honor of Alessandro Volta, with Bell receiving the third grand prize in its history. Since Bell was becoming increasingly affluent, he used his prize money to create endowment funds (the 'Volta Fund') and institutions in and around the United States capital of Washington, D.C.. These included the prestigious 'Volta Laboratory Association' (1880), also known as the Volta Laboratory and as the 'Alexander Graham Bell Laboratory', and which eventually led to the Volta Bureau (1887) as a center for studies on deafness which is still in operation in Georgetown, Washington, D.C. The Volta L

The output looks great. Now let's write a function to generate answers.

In [47]:
def generate_answer(query):
    # tokenize the query to get input_ids
    inputs = tokenizer([query], max_length=1024, return_tensors="pt").to(device)
    # use generator to predict output ids
    ids = generator.generate(inputs["input_ids"], num_beams=2, min_length=20, max_length=40)
    # use tokenizer to decode the output ids
    answer = tokenizer.batch_decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return pprint(answer)

In [48]:
generate_answer(query)

('The first electric power system was built in the early 1800s in the United '
 'States. The first electric power plant was built in the United States in the '
 'early 1900s. The first electric power')


As we can see, the generator used the provided context to answer our question. Let's run some more queries.

In [49]:
query = "How was the first wireless message sent?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

How was the first wireless message sent?<P> In 1880, Bell received the Volta Prize with a purse of 50,000 francs (approximately US$250,000 in today's dollars) for the invention of the telephone from the Académie française, representing the French government. Among the luminaries who judged were Victor Hugo and Alexandre Dumas. The Volta Prize was conceived by Napoleon Bonaparte in 1801, and named in honor of Alessandro Volta, with Bell receiving the third grand prize in its history. Since Bell was becoming increasingly affluent, he used his prize money to create endowment funds (the 'Volta Fund') and institutions in and around the United States capital of Washington, D.C.. These included the prestigious 'Volta Laboratory Association' (1880), also known as the Volta Laboratory and as the 'Alexander Graham Bell Laboratory', and which eventually led to the Volta Bureau (1887) as a center for studies on deafness which is still in operation in Georgetown, Washington, D.C. The Volta Laborato

To confirm that this answer is correct, we can check the contexts used to generate the answer.

In [50]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

In 1880, Bell received the Volta Prize with a purse of 50,000 francs (approximately US$250,000 in today's dollars) for the invention of the telephone from the Académie française, representing the French government. Among the luminaries who judged were Victor Hugo and Alexandre Dumas. The Volta Prize was conceived by Napoleon Bonaparte in 1801, and named in honor of Alessandro Volta, with Bell receiving the third grand prize in its history. Since Bell was becoming increasingly affluent, he used his prize money to create endowment funds (the 'Volta Fund') and institutions in and around the United States capital of Washington, D.C.. These included the prestigious 'Volta Laboratory Association' (1880), also known as the Volta Laboratory and as the 'Alexander Graham Bell Laboratory', and which eventually led to the Volta Bureau (1887) as a center for studies on deafness which is still in operation in Georgetown, Washington, D.C. The Volta Laboratory became an experimental facility devoted t

In this case, the answer looks correct. If we ask a question and no relevant contexts are retrieved, the generator will typically return nonsensical or false answers, like with this question about COVID-19:

In [51]:
query = "where did COVID-19 originate?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

where did COVID-19 originate?<P> In southeast Europe agrarian societies first appeared in the 7th millennium BC, attested by one of the earliest farming sites of Europe, discovered in Vashtëmi, southeastern Albania and dating back to 6,500 BC. Anthropomorphic figurines have been found in the Balkans from 6000 BC, and in Central Europe by c. 5800 BC (La Hoguette). Among the earliest cultural complexes of this area are the Sesklo culture in Thessaly, which later expanded in the Balkans giving rise to Starčevo-Körös (Cris), Linearbandkeramik, and Vinča. Through a combination of cultural diffusion and migration of peoples, the Neolithic traditions spread west and northwards to reach northwestern Europe by around 4500 BC. The Vinča culture may have created the earliest system of writing, the Vinča signs, though archaeologist Shan Winn believes they most likely represented pictograms and ideograms rather than a truly developed form of writing. The Cucuteni-Trypillian culture built enormous s

In [52]:
for doc in context["matches"]:
    print(doc["metadata"]["passage_text"], end='\n---\n')

In southeast Europe agrarian societies first appeared in the 7th millennium BC, attested by one of the earliest farming sites of Europe, discovered in Vashtëmi, southeastern Albania and dating back to 6,500 BC. Anthropomorphic figurines have been found in the Balkans from 6000 BC, and in Central Europe by c. 5800 BC (La Hoguette). Among the earliest cultural complexes of this area are the Sesklo culture in Thessaly, which later expanded in the Balkans giving rise to Starčevo-Körös (Cris), Linearbandkeramik, and Vinča. Through a combination of cultural diffusion and migration of peoples, the Neolithic traditions spread west and northwards to reach northwestern Europe by around 4500 BC. The Vinča culture may have created the earliest system of writing, the Vinča signs, though archaeologist Shan Winn believes they most likely represented pictograms and ideograms rather than a truly developed form of writing. The Cucuteni-Trypillian culture built enormous settlements in Romania, Moldova an

Let’s finish with a final few questions.

In [53]:
query = "what was the war of currents?"
context = query_pinecone(query, top_k=5)
query = format_query(query, context["matches"])
generate_answer(query)

what was the war of currents?<P> Lack of political unity within Greece resulted in frequent conflict between Greek states. The most devastating intra-Greek war was the Peloponnesian War (431–404 BC), won by Sparta and marking the demise of the Athenian Empire as the leading power in ancient Greece. Both Athens and Sparta were later overshadowed by Thebes and eventually Macedon, with the latter uniting the Greek world in the League of Corinth (also known as the Hellenic League or Greek League) under the guidance of Phillip II, who was elected leader of the first unified Greek state in history. <P> Lack of political unity within Greece resulted in frequent conflict between Greek states. The most devastating intra-Greek war was the Peloponnesian War (431–404 BC), won by Sparta and marking the demise of the Athenian Empire as the leading power in ancient Greece. Both Athens and Sparta were later overshadowed by Thebes and eventually Macedon, with the latter uniting the Greek world in the L

In [54]:
query = "who was the first person on the moon?"
context = query_pinecone(query, top_k=10)
query = format_query(query, context["matches"])
generate_answer(query)

who was the first person on the moon?<P> On December 21, 1968, Frank Borman, James Lovell, and William Anders became the first humans to ride the Saturn V rocket into space on Apollo 8. They also became the first to leave low-Earth orbit and go to another celestial body, and entered lunar orbit on December 24. They made ten orbits in twenty hours, and transmitted one of the most watched TV broadcasts in history, with their Christmas Eve program from lunar orbit, that concluded with a reading from the biblical Book of Genesis. Two and a half hours after the broadcast, they fired their engine to perform the first trans-Earth injection to leave lunar orbit and return to the Earth. Apollo 8 safely landed in the Pacific ocean on December 27, in NASA's first dawn splashdown and recovery. <P> On December 21, 1968, Frank Borman, James Lovell, and William Anders became the first humans to ride the Saturn V rocket into space on Apollo 8. They also became the first to leave low-Earth orbit and go

In [55]:
query = "what was NASAs most expensive project?"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

what was NASAs most expensive project?<P> On December 21, 1968, Frank Borman, James Lovell, and William Anders became the first humans to ride the Saturn V rocket into space on Apollo 8. They also became the first to leave low-Earth orbit and go to another celestial body, and entered lunar orbit on December 24. They made ten orbits in twenty hours, and transmitted one of the most watched TV broadcasts in history, with their Christmas Eve program from lunar orbit, that concluded with a reading from the biblical Book of Genesis. Two and a half hours after the broadcast, they fired their engine to perform the first trans-Earth injection to leave lunar orbit and return to the Earth. Apollo 8 safely landed in the Pacific ocean on December 27, in NASA's first dawn splashdown and recovery. <P> On December 21, 1968, Frank Borman, James Lovell, and William Anders became the first humans to ride the Saturn V rocket into space on Apollo 8. They also became the first to leave low-Earth orbit and g

As we can see, the model can generate some decent answers.

#### Add a few more questions

In [61]:
query = "when the world war 2 ended"
context = query_pinecone(query, top_k=3)
query = format_query(query, context["matches"])
generate_answer(query)

when the world war 2 ended<P> On 23 October 1954 – only nine years after Allies (UK, USA and USSR) defeated Nazi Germany ending World War II in Europe – the admission of the Federal Republic of Germany to the North Atlantic Pact was finally decided. The incorporation of West Germany into the organization on 9 May 1955 was described as "a decisive turning point in the history of our continent" by Halvard Lange, Foreign Affairs Minister of Norway at the time. In November 1954, the USSR requested a new European Security Treaty, in order to make a final attempt to not have a remilitarized West Germany potentially opposed to the Soviet Union, with no success. <P> On 23 October 1954 – only nine years after Allies (UK, USA and USSR) defeated Nazi Germany ending World War II in Europe – the admission of the Federal Republic of Germany to the North Atlantic Pact was finally decided. The incorporation of West Germany into the organization on 9 May 1955 was described as "a decisive turning point 